In [11]:
import pandas as pd
import pickle


In [3]:
# ========================================== UTILS

def clean_review(review: str):
    review = review.lower()
    review = review.replace("<br>", "").replace("<br />", "")

    res = ""

    for char in review:
        if (char.isalpha() or char == " "): 
            res += char

    return res



In [10]:
train_reviews_df = pd.read_csv("data/train.csv")
train_reviews_df = train_reviews_df[:25000]
train_reviews_df["review"] = train_reviews_df["review"].apply(clean_review)

test_reviews_df = pd.read_csv("data/test.csv")
test_reviews_df = test_reviews_df[:1000]
test_reviews_df["review"] = test_reviews_df["review"].apply(clean_review)


In [33]:
words_counter = {}

for row_series in train_reviews_df.itertuples():
    added_words = []
    review  = row_series.review
    sentiment  = row_series.sentiment

    for word in review.split(" "):
        if word in added_words:
            continue
        else:
            added_words.append(word)

        if (word not in words_counter):
            words_counter[word] = {"positive": 0, "negative": 0, "total": 0}

        if (sentiment == "positive"):
            words_counter[word]["positive"] += 1
        else:
            words_counter[word]["negative"] += 1

        words_counter[word]["total"] += 1

In [37]:
# ref: https://stackoverflow.com/questions/11218477/how-can-i-use-pickle-to-save-a-dict-or-any-other-python-object


with open('model.pickle', 'wb') as handle:
    a = pickle.dump(words_counter, handle, protocol=pickle.HIGHEST_PROTOCOL)
    print(a)

None


In [34]:


p_pos = len(train_reviews_df[train_reviews_df["sentiment"] == 'positive']) / len(train_reviews_df)
p_neg = len(train_reviews_df[train_reviews_df["sentiment"] == 'negative']) / len(train_reviews_df)

all_words = list(words_counter.keys())

total_pos = len(train_reviews_df[train_reviews_df["sentiment"] == "positive"])
total_neg = len(train_reviews_df[train_reviews_df["sentiment"] == "negative"])


def classificate_review(review):
    global p_neg, p_pos

    p_res_pos = 1
    p_res_neg = 1
    has = {}

    for word in all_words:
        if (word in review):
            has[word] = True
        else:
            has[word] = False

    for word in has:
        if (has[word]):
            p_word_given_positive = (words_counter[word]["positive"] / total_pos)
            p_word_given_negative = (words_counter[word]["negative"] / total_neg)

            p_word = words_counter[word]["total"] / len(train_reviews_df)

            p_positive_given_word = p_word_given_positive / p_word
            p_negative_given_word = p_word_given_negative / p_word
            
            if (p_negative_given_word > 0):
                p_res_neg *= p_negative_given_word

            if (p_positive_given_word > 0):
                p_res_pos *= p_positive_given_word
        else:
            p_n_word_given_positive = 1 - (words_counter[word]["positive"] / total_pos)
            p_n_word_given_negative = 1 - (words_counter[word]["negative"] / total_neg)

            p_n_word = 1 - (words_counter[word]["total"] / len(train_reviews_df))

            p_positive_given_n_word = p_n_word_given_positive / p_n_word
            p_negative_given_n_word = p_n_word_given_negative / p_n_word
            
            if (p_negative_given_n_word > 0):
                p_res_neg *= p_negative_given_n_word
            

            if (p_positive_given_n_word > 0):
                p_res_pos *= p_positive_given_n_word

    p_res_neg *= p_neg
    p_res_pos *= p_pos

    result = "positive"

    if (p_res_pos < p_res_neg):
        result = "negative"

    return result


In [7]:
def test_classification(): 
    right = 0
    wrong = 0

    for row_series in test_reviews_df.itertuples():
        review  = row_series.review
        sentiment = row_series.sentiment

        classification = classificate_review(review)

        if (classification == sentiment):
            right += 1
        else:
            wrong += 1

    print(f"\033[1;37;41m WRONG: {wrong}\033[0m\033[0;37;41m ({(wrong / len(test_reviews_df)) * 100:.2f}%) ")
    print(f"\033[1;37;42m RIGHT: {right}\033[0m\033[0;37;42m ({(right / len(test_reviews_df)) * 100 :.2f}%) ")

test_classification()


 WRONG: 224 (22.40%) 
 RIGHT: 776 (77.60%) 
